In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [ ]:
transform = transforms.Compose([  # Combines multiple preprocessing steps
    transforms.Resize((64, 64)),  # Resize images to make the input of one unique shape
    transforms.ToTensor()         # Convert pixels into tensor
])


In [ ]:
train_data = datasets.ImageFolder(root='dataset/train', transform=transform) # Load Training Data
test_data = datasets.ImageFolder(root='dataset/test', transform=transform)   # Load Test Data


In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True) #Group images to mini batches at a time
test_loader = DataLoader(test_data, batch_size=32)                 # Shuffle= True to randomize the order in each epoch


In [ ]:
import torch.nn as nn
import torch.functional as F

In [ ]:
class DogCatNN(nn.Module):
    def __init__(self):
        super(DogCatNN, self).__init__()
        self.fc1 = nn.Linear(64*64*3, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)


    def forward(self, x):
        x = x.view(x.size(0), -1) # keep the training batch 32 but flatten everything else 64x64x3
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DogCatNN().to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")
